In [31]:
import redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True, db=1)


In [32]:
r.client_info()

{'id': 108,
 'addr': '192.168.65.1:32987',
 'laddr': '10.2.0.2:6379',
 'fd': '18',
 'name': '',
 'age': 0,
 'idle': 0,
 'flags': 'N',
 'db': 1,
 'sub': 0,
 'psub': 0,
 'ssub': '0',
 'multi': -1,
 'qbuf': 26,
 'qbuf-free': 20448,
 'argv-mem': 10,
 'multi-mem': '0',
 'rbs': '16384',
 'rbp': '16384',
 'obl': 0,
 'oll': 0,
 'omem': 0,
 'tot-mem': 37786,
 'events': 'r',
 'cmd': 'client|info',
 'user': 'default',
 'redir': '-1',
 'resp': '2',
 'lib-name': 'redis-py',
 'lib-ver': '5.0.4'}

In [20]:
r.set("foo", 'bar')
r.get("foo")

'bar'

In [33]:
r.get("foo")

In [1]:
from celery import Celery, chain
from search_app import config

task_celery = config.CeleryConf
celery_app = Celery()
celery_app.conf.broker_url = 'redis://localhost:6379/0'
celery_app.conf.result_backend = 'redis://localhost:6379/0'


# resp = celery_app.send_task(config.CeleryConf.task_make_search)

In [2]:
import os
os.environ

environ{'COMMAND_MODE': 'unix2003',
        'CONDA_DEFAULT_ENV': 'projet_certif',
        'CONDA_EXE': '/Users/etiennewagner/miniconda3/bin/conda',
        'CONDA_PREFIX': '/Users/etiennewagner/miniconda3/envs/projet_certif',
        'CONDA_PROMPT_MODIFIER': '(projet_certif) ',
        'CONDA_PYTHON_EXE': '/Users/etiennewagner/miniconda3/bin/python',
        'CONDA_SHLVL': '2',
        'DISPLAY': '/private/tmp/com.apple.launchd.ZMVgpYOTGy/org.xquartz:0',
        'HOME': '/Users/etiennewagner',
        'HOMEBREW_CELLAR': '/opt/homebrew/Cellar',
        'HOMEBREW_PREFIX': '/opt/homebrew',
        'HOMEBREW_REPOSITORY': '/opt/homebrew',
        'INFOPATH': '/opt/homebrew/share/info:',
        'LOGNAME': 'etiennewagner',
        'MANPATH': '/opt/homebrew/share/man::',
        'MallocNanoZone': '0',
        'NVM_CD_FLAGS': '-q',
        'NVM_DIR': '/Users/etiennewagner/.nvm',
        'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined',
        'PATH': '/Users/etiennewagner/miniconda3/envs/projet_ce

In [3]:
os.environ.pop('CELERY_RESULT_BACKEND')
os.environ.pop('CELERY_BROKER_URL')

'redis://redis:6379/0'

In [14]:
result = chain(
    celery_app.signature('get_google_scholar_search_url', kwargs={'query_terms':['vector', 'database']})
    , celery_app.signature('get_webdriver_on_page')
    , celery_app.signature('get_research_pages_on_gs', kwargs={"nb_pages" : 3})
    )(queue="tasks.search")

In [39]:
result.

KeyboardInterrupt: 

In [105]:
import requests
import pandas as pd
from typing import Iterator

def get_dataframe_shuffled(df : pd.DataFrame, column : str | None = None) -> Iterator[pd.Series]:

    rows = df.sample(frac=1).iterrows()

    for row in rows :
        if column :
            yield row[1][column]
        else :
            yield row[1]

class ProxiesHandler():
    free_proxies_providers = ["https://free-proxy-list.net"]

    _retrieved_proxies      = None
    _proxies                = set()
    _used_proxies           = set()
    _not_working_proxies    = set()

    def __init__(self) -> None:
        super.__init__(self)
        self._mimic_header = None

    def get_proxies_elements(self):
        for proxies_provider in self.free_proxies_providers :
            resp = requests.get(proxies_provider)
            proxy_list = pd.read_html(resp.text)[0]
            proxy_list["url"]="http://" + proxy_list["IP Address"] + ":" + proxy_list["Port"].astype(str)
            self._retrieved_proxies = proxy_list[proxy_list["Https"]=="yes"]

    def build_good_proxies(self):
        test_url = "https://httpbin.org/ip"


        for proxy_row in get_dataframe_shuffled(df=self.proxies_retrived):
            proxy_url = proxy_row['url']
            try :
                proxies = {
                    'http'  : proxy_url,
                    'https' : proxy_url
                }
                resp = requests.get(url=test_url, headers=self.mimic_header, proxies=proxies, timeout=2)
                proxy_ip = f"{proxy_row['IP Address']}:{proxy_row['Port']}" if proxy_row['Port'] else proxy_row['IP Address']
                self._proxies.add(proxy_ip)
                self.proxies_retrived.drop(index=self.proxies_retrived[self.proxies_retrived['url']==proxy_url].index)
            except Exception :
                self.proxies_retrived.drop(index=self.proxies_retrived[self.proxies_retrived['url']==proxy_url].index)
            if len(self._proxies) >= 5 :
                break



    @property
    def mimic_header(self):
        if not self._mimic_header :
            resp = requests.get(
                url='https://headers.scrapeops.io/v1/browser-headers',
                params={
                    'api_key': 'eba35a93-5dc6-4c01-971b-57b649ef7b6b',
                    'num_results': '1'}
                )
            self._mimic_header = resp.json()['result'][0]
        
        return self._mimic_header
    
    
    @property
    def proxies_retrived(self):
        if not isinstance(self._retrieved_proxies, pd.DataFrame) :
            self.get_proxies_elements()
        
        if self._retrieved_proxies.empty:
            self.get_proxies_elements()
        
        return self._retrieved_proxies
    
    @property
    def proxies(self):
        if not self._proxies :
            self.build_good_proxies()
        return self._proxies
    
    @property
    def proxy(self):

        return self.proxies.pop()

In [107]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome, ChromeOptions
import uuid
import os
from pathlib import Path
import requests
import pandas as pd
from random import choice
from typing import Iterator
import logging

lg = logging.getLogger(__name__)
class ScrapingDriver(ProxiesHandler):
    AGENT_LIST = [
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
        "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:24.0) Gecko/20100101 Firefox/24.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/91.0.4472.114 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0"
        ]
    
    def __init__(self) -> None:
        # super.__init__(self)
        pass

    def get_driver(self, url : str) -> tuple[Chrome, Path]:
        # lg.info("Creation d'un moteur de scraping.")
        service = Service(executable_path='/usr/bin/chromedriver')
        chrome_options = ChromeOptions()

        # temp_folder = Path(os.getenv("TEMP_EMPL"))
        # temp_folder.mkdir(parents=True, exist_ok=True)
        temp_folder = Path('./temp_' + str(uuid.uuid4()))
        temp_folder.mkdir(parents=True, exist_ok=True)

        prefs = {
            "download.default_directory" : str(temp_folder.absolute()),
            'download.prompt_for_download': False,
            'plugins.always_open_pdf_externally': True,
        }

        chrome_options.add_argument("--headless=new")
        # chrome_options.add_argument('--disable-blink-features')
        chrome_options.add_argument("--incognito")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument(f'--proxy-server={self.proxy}')
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

        chrome_options.add_experimental_option('prefs', prefs)

        

        driver = Chrome(
            options=chrome_options,
            service=service
            )
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": choice(self.AGENT_LIST)})
        driver.get(url=url)

        return driver, temp_folder

In [108]:
ph = ScrapingDriver()

In [109]:
ph.proxy

/var/folders/yp/9gm2w_f53xj6vsx7dgtz95_c0000gn/T/ipykernel_11634/1118225445.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  proxy_list = pd.read_html(resp.text)[0]


KeyError: 'pop from an empty set'

In [52]:
def build_good_proxies(proxies : set):
    test_url = "https://httpbin.org/ip"


    for proxy_row in get_dataframe_shuffled(df=ph.proxies_retrived):
        proxy_url = proxy_row['url']
        try :
            proxies = {
                'http'  : proxy_url,
                'https' : proxy_url
            }
            resp = requests.get(url=test_url, headers=ph.mimic_header, proxies=proxies, timeout=10)
            print(resp.json()['origin'])
            proxy_ip = f"{proxy_row['IP Address']}:{proxy_row['Port']}" if proxy_row['Port'] else proxy_row['IP Address']
            proxies.add(proxy_ip)
            ph.proxies_retrived.drop(index=ph.proxies_retrived[ph.proxies_retrived['url']==proxy_url].index)
        except Exception :
            print('error')
            ph.proxies_retrived.drop(index=ph.proxies_retrived[ph.proxies_retrived['url']==proxy_url].index)
        if len(proxies) >= 5 :
            break
essai = set()
build_good_proxies(proxies=essai)

error
134.35.34.104
error
error
209.15.117.148
error
error
error
35.247.44.173
error
error
error
error


In [39]:
ph.mimic_header

{'upgrade-insecure-requests': '1',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36 Edge/15.15063',
 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
 'sec-ch-ua': 'Google Chrome;v="95", "Chromium";v="95", ";Not A Brand";v="99"',
 'sec-ch-ua-mobile': '?0',
 'sec-ch-ua-platform': 'Windows',
 'sec-fetch-site': 'none',
 'sec-fetch-mod': '',
 'sec-fetch-user': '?1',
 'accept-encoding': 'gzip',
 'accept-language': 'en-US,es;q=0.5'}

In [44]:
proxies = {
                'http'  : 'http://52.16.232.164:3128',
                'http' : 'http://52.16.232.164:3128'
            }
test_url = "https://httpbin.org/ip"
resp = requests.get(url=test_url, headers=ph.mimic_header, proxies=proxies)

In [47]:
resp.content

b'{\n  "origin": "82.64.147.13"\n}\n'

In [40]:
celery_app.control.ping()

[{'celery@c62fff0ba91a': {'ok': 'pong'}}]

In [18]:
resp = celery_app.send_task(name=config.CeleryConf.task_make_search,
                            kwargs={
                                "prompt" : 'What is vector database ?',
                                "search_type" : 'api',
                                "search_platform" : 'google_scholar',
                            },
                            queue="tasks.search")

resp.id

'707fd22b-5243-4474-be4e-62516ddc33d8'

In [28]:
from search_app.core.databases.elasticsearch_backend import ESHandler
os.environ["ES_VM_CERTIF_PATH"] = '/Users/etiennewagner/Documents/Reconversion/Licence IA/2eme_annee/projet_certif.nosync/certifs/es/'
es = ESHandler()

In [31]:
es.es.license.get()

ObjectApiResponse({'license': {'status': 'active', 'uid': '8dacd13d-808a-4b23-9da4-104a95d6d4a5', 'type': 'trial', 'issue_date': '2024-06-11T14:16:51.339Z', 'issue_date_in_millis': 1718115411339, 'expiry_date': '2024-07-11T14:16:51.339Z', 'expiry_date_in_millis': 1720707411339, 'max_nodes': 1000, 'max_resource_units': None, 'issued_to': 'iserarchapp-cluster', 'issuer': 'elasticsearch', 'start_date_in_millis': -1}})

In [19]:
a = set()
if not a :
    print("heay")

heay


In [33]:
resp = celery_app.send_task(name='get_gs_url',
                            kwargs={
                                "query_terms" : ['vector', 'database']
                            },
                            queue="tasks.search")

resp.id

'd2fb31e3-616b-4cf0-8ef4-6dee5ad030ff'

In [53]:
resp.state

'SUCCESS'

In [37]:
resp.result[0][0]

'a6155793-6b8a-4469-ae94-aaffea375112'

In [58]:
resp.forget()

In [95]:
result = celery_app.AsyncResult(id='2b6c7996-1b30-4f7e-9826-92e847222bdb')
result.result

In [42]:
result.result

In [46]:
resp.forget()

In [129]:
a = list(resp.collect())

KeyboardInterrupt: 

In [101]:
a = [ ele for ele in resp.collect()]

KeyboardInterrupt: 

In [93]:
result = celery_app.AsyncResult(id='5b1486a1-edc0-407f-b385-bf3a5a92f1c6')

In [94]:
result.result

[]

In [14]:
result.revoke(terminate=True)

In [18]:
i = celery_app.control.inspect()

In [24]:
i.reserved()

{'celery@c62fff0ba91a': []}

In [19]:
i.scheduled()

{'celery@c62fff0ba91a': []}

In [136]:
celery_app.control.revoke('34a7379e-1009-4477-8538-4b49af0fe451')

In [16]:
celery_app.control.purge()

0

In [20]:
i.reserved()

{'celery@c62fff0ba91a': []}

In [21]:
result.status

'FAILURE'

In [83]:
resp.get()

KeyboardInterrupt: 